### Google Analytics 4 and BigQuery with Pandas
#### [GA4 Public Dataset](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=ga4_obfuscated_sample_ecommerce&t=events_20210131&page=table)


In [2]:
# Uncomment the below lines and run them if you haven't installed the module before
#%pip install google
#%pip install db-dtypes
#/\ NOT WORKING
#!pip install google-cloud-bigquery google-auth pandas


Note: you may need to restart the kernel to use updated packages.



In [9]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Use a service account for authentication onto GCP
credentials = service_account.Credentials.from_service_account_file('./secrets/ecommercedata-413417-6497dae93c8d.json')

# The project id to which the service account has access
project_id = 'ecommercedata-413417'

# Establish a connection to the client
client = bigquery.Client(credentials= credentials, project=project_id)

# Perform a query
query = client.query(
"""
SELECT *
FROM
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20201101`
""")

#Convert output into a pandas dataframe
df = query.to_dataframe()
df

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,user_ltv,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items
0,20201101,1604192073036492,page_view,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,2351439526,<NA>,None,1005694.5834348403,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Southe...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
1,20201101,1604192073036492,first_visit,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,2351439526,<NA>,None,1005694.5834348403,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Southe...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
2,20201101,1604192082302672,user_engagement,"[{'key': 'dclid', 'value': {'string_value': No...",<NA>,NaN,-4853805495,<NA>,None,1005694.5834348403,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Southe...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
3,20201101,1604192073036492,session_start,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,2351439526,<NA>,None,1005694.5834348403,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Southe...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
4,20201101,1604202750089608,user_engagement,"[{'key': 'all_data', 'value': {'string_value':...",<NA>,NaN,7473184003,<NA>,None,1013442.5000387623,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'mobile', 'mobile_brand_name': 'S...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31267,20201101,1604191178435076,view_item,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,-4765920043,<NA>,None,8371382448.1492318005,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'cpc', 'name': '<Other>', 'source':...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...","[{'item_id': 'GGOEGXXX1038', 'item_name': 'Goo..."
31268,20201101,1604236810906150,view_item,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,4629187654,<NA>,None,8665067596.5834536181,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...","[{'item_id': 'GGOEGDNL108699', 'item_name': 'G..."
31269,20201101,1604236804332730,view_item,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,6779564612,<NA>,None,8665067596.5834536181,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...","[{'item_id': 'GGOEGDWQ123399', 'item_name': 'G..."
31270,20201101,1604220772353910,view_item,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,594470926,<

In [5]:
# Show unique values from event_name column
name_values = list(set(df['event_name']))#.tolist()
print(name_values)

# Show how many times each value from event
name_counts = df['event_name'].value_counts()
print(name_counts)

['page_view', 'view_item', 'user_engagement', 'add_to_cart', 'first_visit', 'view_item_list', 'select_promotion', 'scroll', 'session_start', 'view_promotion', 'view_search_results', 'begin_checkout', 'click', 'purchase', 'add_shipping_info', 'add_payment_info']
event_name
page_view              9767
user_engagement        8667
view_item              3380
scroll                 3040
session_start          2594
first_visit            1946
view_promotion         1263
view_search_results     194
begin_checkout          127
add_shipping_info       127
add_payment_info         73
select_promotion         57
purchase                 14
click                    11
view_item_list            8
add_to_cart               4
Name: count, dtype: int64


In [6]:
column_name = 'event_date'

# time frame for data
name_values = list(set(df[column_name]))#.tolist()
print(name_values)

# Show how many times each value from event
name_counts = df[column_name].value_counts()
print(name_counts)

# check for null values
null_values = df[column_name].isnull().any()
print(null_values)

['20201101']
event_date
20201101    31272
Name: count, dtype: int64
False


In [7]:
column_name = 'event_timestamp'

# time frame for data
# name_values = list(set(df[column_name]))#.tolist()
# print(name_values)

# Show how many times each value from event
name_counts = df[column_name].value_counts()
print(name_counts)

# check for null values
null_values = df[column_name].isnull().any()
print(null_values)

event_timestamp
1604210589898438    4
1604237154913974    4
1604231168859166    4
1604261495168930    4
1604274627038858    4
                   ..
1604252692732505    1
1604252603713018    1
1604214574422023    1
1604235641006094    1
1604220772353910    1
Name: count, Length: 21452, dtype: Int64
False


In [11]:
column_name = 'event_params'

print(df[column_name][0])
print(df[column_name][1])

# check for null values
null_values = df[column_name].isnull().any()
print(null_values)

[{'key': 'page_title', 'value': {'string_value': 'Drinkware | Lifestyle | Google Merchandise Store', 'int_value': None, 'float_value': None, 'double_value': None}}
 {'key': 'all_data', 'value': {'string_value': None, 'int_value': None, 'float_value': None, 'double_value': None}}
 {'key': 'page_location', 'value': {'string_value': 'https://shop.googlemerchandisestore.com/Google+Redesign/Lifestyle/Drinkware', 'int_value': None, 'float_value': None, 'double_value': None}}
 {'key': 'ga_session_number', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}
 {'key': 'clean_event', 'value': {'string_value': 'gtm.js', 'int_value': None, 'float_value': None, 'double_value': None}}
 {'key': 'engaged_session_event', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}
 {'key': 'dclid', 'value': {'string_value': None, 'int_value': None, 'float_value': None, 'double_value': None}}
 {'key': 'session_engaged', 'value': {'str

In [18]:
# check if any columns with no values in rows - check .any and .all
columns_with_null = df.columns[df.isnull().all()].tolist()
print(columns_with_null)

# these columns are completely empty: ['event_previous_timestamp', 'event_server_timestamp_offset', 'user_id', 'app_info', 'event_dimensions']
# so, I know now to use pseudo_user_id, not user_id

['event_previous_timestamp', 'event_server_timestamp_offset', 'user_id', 'app_info', 'event_dimensions']


In [23]:
column_name = 'user_pseudo_id'
# check if pseudo_user_id has duplicates
duplicates = df.duplicated(subset=[column_name])
print(duplicates)

name_counts = df[column_name].value_counts()
print(name_counts)

0        False
1         True
2         True
3         True
4        False
         ...  
31267     True
31268     True
31269     True
31270     True
31271     True
Length: 31272, dtype: bool
user_pseudo_id
58162644.7624663170    306
5120145.3258256986     304
17163314.1472925430    303
4712697.9761669562     303
2446406.6227301483     303
                      ... 
8525578.1317471666       2
95983775.2884447790      2
7563940.4704773791       2
72581747.6734625536      1
1099668.0624711937       1
Name: count, Length: 2365, dtype: int64


In [ ]:
# privacy info